Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 1: Set Covering

First lab + peer review. List this activity in your final report, it will be part of your exam.

## Task

Given a number $N$ and some lists of integers $P = (L_0, L_1, L_2, ..., L_n)$, 
determine is possible $S = (L_{s_0}, L_{s_1}, L_{s_2}, ..., L_{s_n})$
such that each number between $0$ and $N-1$ appears in at least one list

$$\forall n \in [0, N-1] \ \exists i : n \in L_{s_i}$$

and that the total numbers of elements in all $L_{s_i}$ is minimum. 

## Instructions

* Create the directory `lab1` inside the course repo (the one you registered with Andrea)
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)
* Use `problem` to generate the problems with different $N$
* In the `README.md`, report the the total numbers of elements in $L_{s_i}$ for problem with $N \in [5, 10, 20, 100, 500, 1000]$ and the total number on $nodes$ visited during the search. Use `seed=42`.
* Use `GitHub Issues` to peer review others' lab

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

**Deadline**

* Sunday, October 16th 23:59:59 for the working solution
* Sunday, October 23rd 23:59:59 for the peer reviews

In [1]:
from operator import index
import random
import sys
from time import time
import numpy as np
import logging
from collections import Counter

N_total = [5, 10, 14, 20] #, 100, 500, 1000]

In [2]:
def problem(N, seed=42):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [3]:
class State():
    """Class for states in A* alghoritm"""

    def __init__(self, state, number_found, g, h, remaining_list = []):
        self.g = g
        self.h = h
        self.f = g + h

        self.remaining_list = remaining_list
        self.state = state
        self.number_found = number_found
        

In [4]:
def preprocessing(list_of_lists):

    list_of_lists =  sorted(list_of_lists, key = lambda l : len(l))

    return list_of_lists

In [5]:
def greedy_best_first(N):
    goal = set(range(N))
    solution = []
    start_problem = preprocessing(problem(N))
    
    number_found = set()
    n_of_visited_nodes = 0
    
    while goal > number_found:
        n_of_visited_nodes+=1    
        element = start_problem.pop(0)
        if set(element) | number_found > number_found: 
            number_found = set(element) | number_found
            solution.append(element)
    
    print(f"\nSolution using Greedy Best-First algotithm with N = {N} =>\n\t W = {sum(len(element) for element in solution)} \n\t N of VISITED NODES = {n_of_visited_nodes}")
    # print(f"Solution {solution}")


In [6]:
#Actual Cost
def g(solution):
    cnt = Counter()
    
    
    state = [tuple(sol) for sol in solution]

    cnt.update(sum((e for e in state), start=()))
    # return sum(cnt[c] - 1 for c in cnt if cnt[c] > 1) + sum(len(element)  for element in solution) 
    # print(f"A: {sum(len(element) for element in solution)} = {sum(cnt[c] - 1 for c in cnt if cnt[c] > 1)} + {sum(cnt[c] == 1 for c in cnt)}")
    # return sum(len(element) for element in solution) - sum(cnt[c] == 1 for c in cnt)
    return sum(len(element) for element in solution) + len(solution[-1])

In [7]:
#Heuristic Cost
def h(N, number_found):
    return (N - (len(number_found)))

In [8]:
def a_star(N):

    n_of_visited_nodes = 0
    start_problem = problem(N)
    goal = set(range(N))
    state_list = []

    open_states = []
    
    for ind, element in enumerate(start_problem):
        n_of_visited_nodes += 1
        state_list.append(element)
        temp_state = State(state_list, set(element) ,g(state_list), h(N, set(element)))
        open_states.append(temp_state)
        state_list = []
    

    while True:
    
        ind = 0
        current_state = open_states[ind]

        for indice, open_state in enumerate(open_states):

            if open_state.f < current_state.f:
                current_state = open_state
                ind = indice
        
        open_states.pop(ind)
    
        number_found = current_state.number_found      

        if number_found >= goal:
            print("solution :", current_state.state)
            print("W: ", sum(len(element) for element in current_state.state))
            return

        curr_state = current_state.state
        
        for element in start_problem:
            
            #Needed to not have duplicates
            if element not in current_state.state:
                n_of_visited_nodes += 1
                state_list = curr_state.copy()
                state_list.append(element)
                number_found = set(element) | current_state.number_found

                temp_state = State(state_list, number_found ,g(state_list), h(N, number_found))
                
                if temp_state.number_found >= goal:
                    print(f"\nSolution using A* algotithm with N = {N} =>\n\t W = {sum(len(element) for element in temp_state.state)} \n\t N of VISITED NODES = {n_of_visited_nodes}")
                    print(f"Solution {temp_state.state}")
                    return

                open_states.append(temp_state)



In [9]:
# for n in N_total:
#     st = time()
#     greedy_best_first(n)
#     et = time()
#     print(f"Elapsed Time = {et-st}seconds")

In [10]:
for n in N_total:
    st = time()
    a_star(n)
    et = time()
    print(f"Elapsed Time = {et-st}seconds")


Solution using A* algotithm with N = 5 =>
	 W = 5 
	 N of VISITED NODES = 4558
Solution [[0], [1], [4], [2, 3]]
Elapsed Time = 0.11469244956970215seconds

Solution using A* algotithm with N = 10 =>
	 W = 10 
	 N of VISITED NODES = 27231
Solution [[0, 4], [9, 6], [2, 5], [8, 1, 3, 7]]
Elapsed Time = 1.6015267372131348seconds
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\marco\anaconda3\envs\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\marco\AppData\Local\Temp\ipykernel_11684\1400987408.py", line 3, in <cell line: 1>
    a_star(n)
  File "C:\Users\marco\AppData\Local\Temp\ipykernel_11684\895157113.py", line -1, in a_star
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\marco\anaconda3\envs\venv\lib\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\marco\anaconda3\envs\venv\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\marco\anaconda3\envs\venv\lib\site-packages\IPython\core\ultratb.py", line 1012, in structured_traceback